In [ ]:
import pandas as pd
import numpy as np
import pickle
import scipy
import gc
from sklearn.decomposition import PCA, TruncatedSVD,KernelPCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder, OneHotEncoder,Normalizer
import seaborn as sns
from umap import UMAP
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
root_path = "../../../input/open-problems-raw-counts/"

## Inputs

In [ ]:
def tfidf(X):
    r"""
    TF-IDF normalization (following the Seurat v3 approach)
    Parameters
    ----------
    X
        Input matrix
    Returns
    -------
    X_tfidf
        TF-IDF normalized matrix
    """
    idf = X.shape[0] / X.sum(axis=0)
    if scipy.sparse.issparse(X):
        tf = X.multiply(1 / X.sum(axis=1))
        return tf.multiply(idf)
    else:
        tf = X / X.sum(axis=1, keepdims=True)
        return tf * idf

In [ ]:
train = pd.read_pickle(f"{root_path}train_multi_inputs_raw.pkl")
train_indexes = train.index
train = scipy.sparse.csr_matrix(train)
train.shape

In [ ]:
test = pd.read_pickle(f"{root_path}test_multi_inputs_raw.pkl")
test = scipy.sparse.csr_matrix(test)
test.shape

In [ ]:
all = scipy.sparse.vstack([train,test])
del train,test
gc.collect()
all.shape

In [ ]:
all = tfidf(all)
all = np.log1p(all * 1e4)

In [ ]:
pure_tsvd = TruncatedSVD(n_components=512, random_state=42)
all_tsvd = pure_tsvd.fit_transform(all)
print(pure_tsvd.explained_variance_ratio_.sum())

In [ ]:
np.savez("../new_multi_train_tsvd.npz",all_tsvd[:105933])
np.savez("../new_multi_test_tsvd.npz",all_tsvd[105933:])

## Targets

In [ ]:
targets = pd.read_hdf("../../../input/open-problems-multimodal/train_multi_targets.h5")
targets.shape

In [ ]:
targets = targets.values
targets -= targets.mean(axis=1, keepdims=True)
targets /= targets.std(axis=1, ddof=1, keepdims=True)

In [ ]:
pure_tsvd_ = TruncatedSVD(n_components=1024, random_state=42)
targets_other = pure_tsvd_.fit_transform(targets)
print(pure_tsvd_.explained_variance_ratio_.sum())

In [ ]:
with open("./new_multi_tsvd_1024.pkl","wb") as f:
    pickle.dump(pure_tsvd_,f)

In [ ]:
np.savez("new_multi_target_tsvd.npz",targets_other)

In [ ]:
np.savez_compressed("new_multi_target.npz",targets)

## Get new and old indexs

In [ ]:
train_ori_col= np.load("../../../input/multimodal-single-cell-as-sparse-matrix/train_multi_inputs_idxcol.npz",allow_pickle=True)["index"]

In [ ]:
in_ori = [i for i in tqdm(train_indexes) if i in train_ori_col]
len(in_ori)  # 105868

In [ ]:
in_ori_idx = [np.where(train_indexes==i)[0].item() for i in tqdm(in_ori)]
in_ori_ori_idx = [np.where(train_ori_col==i)[0].item() for i in tqdm(in_ori)]

In [ ]:
with open("multi_in_ori_raw_idx.pkl","wb") as f:
    pickle.dump(in_ori_idx,f)
with open("multi_in_ori_ori_idx.pkl","wb") as f:
    pickle.dump(in_ori_ori_idx,f)